<h1><center>Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h3><center>Problem 1</center></h3>
<h3><center>Scraping Data from Wikipedia</center></h3>

<h4>The URL to the page on Wikipedia</h4>

In [17]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

<h4>Import Libraries</h4>

In [166]:
# Imports for Problem 1
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Imports for Problem 2


<h4>Request page from URL</h4>

In [19]:
request = requests.get(URL)
request = request.text

<h4>Create a Beautiful Soup Object and extract tables</h4>

In [20]:
soup = BeautifulSoup(request, 'xml')
table = soup.find('table')
print(f'Type : {type(table)}')
print(f'Length : {len(table)}')

Type : <class 'bs4.element.Tag'>
Length : 2


<h4>Extract all the data from the table into python lists</h4>

In [145]:
# Search all the postcode, borough, neighborhood 
PostalCode = []
Borough = []
Neighborhood = []
for tr in table.find_all('tr'):
    for b in tr.find_all('b'):
        PostalCode.append(b.text)
    for span in tr.find_all('span'):
        span = span.text
        borough = ""
        neigh = re.search('\((.*?)\)', span)
        if neigh is not None:
            neigh = neigh.group()
            borough = span.replace(neigh,"")
            neigh = neigh.replace("/", ",")
            brackets = ('()') 
            for x in neigh:
                if x in brackets:
                    neigh = neigh.replace(x, "") 
        else:
            neigh = 'Not Assigned'
        if borough is "":
            borough = 'Not Assigned'
        Borough.append(borough)
        Neighborhood.append(neigh)
        

print(f'Length of PostalCode : {len(PostalCode)}')
print(f'Length of Neighborhood : {len(Neighborhood)}')
print(f'Length of Borough : {len(Borough)}')

Length of PostalCode : 180
Length of Neighborhood : 180
Length of Borough : 180


<h4>Create a pandas Dataframe</h4>

In [146]:
df = pd.DataFrame({
    'PostalCode' : PostalCode,
    'Borough' : Borough,
    'Neighborhood' : Neighborhood
})

<h4>Display first 5 rows of dataframe</h4>

In [140]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not Assigned,Not Assigned
1,M2A,Not Assigned,Not Assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


<h4>Remove all rows with Borough is 'Not Assigned'</h4>

In [141]:
pbn_df = df[df['Borough'] != 'Not Assigned']

<h4>Reset Index</h4>

In [147]:
pbn_df = pbn_df.reset_index(drop=True)
pbn_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<h4>Display Shape</h4>

In [144]:
pbn_df.shape

(103, 3)

<h3><center>Problem 2</center></h3>
<h3><center>Getting Latitude and Longitude</center></h3>

<h4>Import Libraries</h4>

In [157]:
# All import are performed at one place on the top of the notebook

In [172]:
pll_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [174]:
pll_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
pll_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
pbnll_df = pd.merge(pbn_df, pll_df, on='PostalCode')

In [183]:
pbnll_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
